In [1]:
from scipy.spatial import ConvexHull
import numpy as np
from math import sqrt
from itertools import combinations

"""
Two boundary points will never connect apart from current edges

Subtract outer radius from outer_pts/connect points connection, and subtract associated
distance from each entry in Short_List

Continue to include outer points in each shortest iteration, but eliminate key-point and
include current_edges restriction.
"""

"""
    Start with a convex hull whose points are OP and with the interior points, IP.

    Find Euclidean distances between every pair of points.

    The shortest distance for any OP to an IP is the current outer radius, "OR".

    WHILE all IP are NOT connected by two edges,

        a) IF the distance from that now connected IP, namely IP_1, is longer than any
    distance from IP_1 to another unconnected IP or another edge (as edges can
    connect to each other)
             i) THEN connect to that point AND repeat step 4.

        b) ELSE, all newly connected IP_n become part of the OP set, AND CONTINUE to
    next smallest "OR" (including the connected IP_n).

        c) Whenever inner edges from two OP, through any number of IP_n, connect, DELETE
    the edge between those OP (including IP_n that are now OP).
"""

In [2]:
allPoints = []

n=15 # number of points
N=10 # +/- range of points 

for i in range(n):
    points = (float(np.random.randint(-N,N)),float(np.random.randint(-N,N)))
    allPoints.append(points)
# creates list of random points

allPoints = list(set(allPoints)) #takes out possible duplicates

print("allPoints:",allPoints,"\n")

allPoints: [(-8.0, 9.0), (-9.0, -1.0), (-3.0, -2.0), (9.0, 5.0), (-6.0, -4.0), (6.0, -2.0), (-3.0, -10.0), (-9.0, 3.0), (3.0, -4.0), (-2.0, -2.0), (-6.0, 7.0), (3.0, 7.0), (5.0, -3.0), (-6.0, -7.0)] 



In [3]:
class TravelPoly:
    def __init__(self,cls):
        self.OP = [allPoints[i] for i in ConvexHull(allPoints).vertices]
        print("OP:",self.OP,"\n")
        
        self.IP = [x for x in allPoints if x not in self.OP]
        print("IP:",self.IP,"\n"*2)

        self.currEdges = []
        self.outerDistsDict = {}
        self.innerDistsList = []

        self.currOR = 0
        self.currIR = 0
    
        cls.getCurrEdges(self,cls)
        cls.getOuterDistsDict(self,cls)
        cls.getInnerDistsList(self,cls)

    def algorithm(self,cls):
        '''
        TODO WHILE there are IP (inner points) not connected by two edges
        '''
        cls.getMinOR(self,cls)
        self.listIR = []
        cls.getListIR(self,cls,self.listOR)

        self.listIR.sort(key=lambda tup: tup[1][1])
        print("listIR:",self.listIR)
        try:    
            self.currIR = self.listIR[0]
        except:
            pass


    @staticmethod
    def distance(point1, point2):
        return sqrt((point1[0]-point2[0])**2+(point1[1]-point2[1])**2)    
        
    def getCurrEdges(self,cls):    
        for i in range(len(self.OP)):
            self.currEdges = [(self.OP[i],self.OP[(i+1)%len(self.OP)]) for i in range(len(self.OP))] 
        print("currEdges:",self.currEdges,"\n"*2)

    def getOuterDistsDict(self,cls): 
        print("outerDistsDict:")
        for a in self.OP:
            temp = []
            for b in self.IP:
                temp.append((b,cls.distance(a,b)))
            temp.sort(key=lambda tup: tup[1])
            print(a,":",temp,"\n")
            self.outerDistsDict.update({a:temp})
            
    def getInnerDistsList(self,cls):
        print("\n""innerDistsList:")
        for (a,b) in combinations(self.IP,2):
            self.innerDistsList.append(((a,b),cls.distance(a,b)))
        
        self.innerDistsList.sort(key=lambda tup: tup[1])
        print(self.innerDistsList)

    def getMinOR(self,cls):
        self.listOR = [] 
        min_value = float("inf")
        for (k,v) in self.outerDistsDict.items():
            if v[0][1] - self.currIR == min_value:
                cls.checkForMultiOR(self,k,v,min_value,self.listOR)
            elif v[0][1] - self.currIR < min_value:
                min_value = v[0][1] - self.currIR
                self.listOR = []
                cls.checkForMultiOR(self,k,v,min_value,self.listOR)
        
        self.currOR = self.listOR[0][1][1]
        print("\n"*2,"minOR_list:",self.listOR,"\n""currOR:",self.currOR,"\n"*2)
        for (key,val) in self.listOR: # Removes minOR_list values from outerDistsDict dictionary
            self.outerDistsDict[key] = self.outerDistsDict[key].remove(val)
 
    def checkForMultiOR(self,key,val,min_value,radiusList):
        radiusList.append((key,(val[0][0],val[0][1])))
        for i in range(len(val)-1):
            if val[i+1][1] == min_value:
                radiusList.append((key,(val[i+1][0],val[i+1][1])))
            else:
                break
        
    def checkForIR(self,radiusList):       
        temp = []
        for ((i,j),k) in self.innerDistsList:
            for (a,(b,c)) in radiusList:
                if k <= c:
                    if b == j:
                        if not temp:
                            temp.append((b,(i,k)))
                        elif k == temp[0][1]:
                            temp.append((b,(i,k)))
                        elif k < temp[0][1]:
                            temp = []
                            temp.append((b,(i,k)))
                    elif b == i:
                        if not temp:
                            temp.append((b,(j,k)))
                        elif k == temp[0][1]:
                            temp.append((b,(j,k)))
                        elif k < temp[0][1]:
                            temp = []
                            temp.append((b,(j,k)))
                else:
                    return temp                  

    def getListIR(self,cls,temp):
        temp = cls.checkForIR(self,self.listOR)
        self.listIR += temp
        print("temp:",temp,"\n")
        for (a,(b,c)) in temp: # Removes temp values from innerDistsList dictionary
            try:
                self.innerDistsList.remove(((a,b),c))
            except:
                self.innerDistsList.remove(((b,a),c))
        
        while temp:
            temp = cls.getListIR(self,cls,temp)

    def updateOP(self,listName):
        for (a,(b,c)) in listName:
            pass
    
    def getTouchedPoints(self):
        pass

    def updateEdges(self):
        pass

    def breakEdges(self):
        pass

In [4]:
TravelPoly(TravelPoly).algorithm(TravelPoly)

OP: [(-9.0, -1.0), (-6.0, -7.0), (-3.0, -10.0), (5.0, -3.0), (6.0, -2.0), (9.0, 5.0), (3.0, 7.0), (-8.0, 9.0), (-9.0, 3.0)] 

IP: [(-3.0, -2.0), (-6.0, -4.0), (3.0, -4.0), (-2.0, -2.0), (-6.0, 7.0)] 


currEdges: [((-9.0, -1.0), (-6.0, -7.0)), ((-6.0, -7.0), (-3.0, -10.0)), ((-3.0, -10.0), (5.0, -3.0)), ((5.0, -3.0), (6.0, -2.0)), ((6.0, -2.0), (9.0, 5.0)), ((9.0, 5.0), (3.0, 7.0)), ((3.0, 7.0), (-8.0, 9.0)), ((-8.0, 9.0), (-9.0, 3.0)), ((-9.0, 3.0), (-9.0, -1.0))] 


outerDistsDict:
(-9.0, -1.0) : [((-6.0, -4.0), 4.242640687119285), ((-3.0, -2.0), 6.082762530298219), ((-2.0, -2.0), 7.0710678118654755), ((-6.0, 7.0), 8.54400374531753), ((3.0, -4.0), 12.36931687685298)] 

(-6.0, -7.0) : [((-6.0, -4.0), 3.0), ((-3.0, -2.0), 5.830951894845301), ((-2.0, -2.0), 6.4031242374328485), ((3.0, -4.0), 9.486832980505138), ((-6.0, 7.0), 14.0)] 

(-3.0, -10.0) : [((-6.0, -4.0), 6.708203932499369), ((-3.0, -2.0), 8.0), ((-2.0, -2.0), 8.06225774829855), ((3.0, -4.0), 8.48528137423857), ((-6.0, 7.0), 1